In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('/kaggle/input/alzheimers-prediction-dataset-global/alzheimers_prediction_dataset.csv')
data.head()
data

**DATA ANALYSIS**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns



country_counts = data['Country'].value_counts()
unique_countries = data['Country'].unique().tolist()

print(country_counts)
print(unique_countries)

sns.countplot(x=data['Genetic Risk Factor (APOE-ε4 allele)'], hue = data['Alzheimer’s Diagnosis'])

    

    

The previous graph made it clear that if someone has genetic risk fastor its a higher chance that the person will get alzhimers

In [ ]:
plt.figure(figsize=(15, 6))


column_indices = [0, 2, 5, 6, 7]


for idx in column_indices:
    column_name = data.columns[idx]  # Get the column name by index
    sns.countplot(x=data[column_name], hue=data['Alzheimer’s Diagnosis'])
    
    # Set labels and title for clarity
    plt.xlabel(column_name)
    plt.title(f'Countplot of {column_name} vs Alzheimer’s Diagnosis')
    
    # Show the plot
 






plt.show()
for country in unique_countries :
    sns.displot(data[(data['Alzheimer’s Diagnosis'] == 'Yes') & (data['Country'] == country)]['Age'])
    plt.xlabel(country)

    


The previous analysis was meant to understand how the disease is varying country wise with age. And a very similar pattern was found with some outliers which can be ignored safely.
The main intention behind this was to know if the country have any significant effect on disease as varied with age.

In [ ]:
sns.countplot(x=data['Diabetes'], hue=data['Alzheimer’s Diagnosis'])
plt.show()

a = ((data['Alzheimer’s Diagnosis'] == 'Yes') & (data['Diabetes'] == 'Yes')).sum() / (data['Alzheimer’s Diagnosis'] == 'Yes').sum()
#b= ((data['Alzheimer’s Diagnosis'] == 'Yes') & (data['Diabetes'] == 'No')).sum() / (data['Alzheimer’s Diagnosis'] == 'Yes').sum()

print(a)
print(b)

The previous part showed that having diabetes do no have much effect on the disease

In [ ]:
sns.boxplot(x=data['Alzheimer’s Diagnosis'], y=data['BMI'])

In [ ]:
sns.boxplot(x=data['Alzheimer’s Diagnosis'], y=data['Age'])

In [ ]:
sns.countplot(x= data['Gender'], hue=data['Alzheimer’s Diagnosis'])

In [ ]:
sns.boxplot(x="Alzheimer’s Diagnosis", y="Age", hue="Gender", data=data)

In [ ]:
sns.countplot(x= data['Gender'], hue=data['Alzheimer’s Diagnosis'])

The previous graphs showed that only age is a major factor in determining if a person have the disease or not.

**Feature engineering**

In [ ]:
data

In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply Label Encoding to the categorical output (removing any trailing spaces)
data['Alzheimer’s Diagnosis'] = label_encoder.fit_transform(data['Alzheimer’s Diagnosis'].str.strip())

# Check the mapping of original labels to encoded values


# Define the order of categories
categories = [['Never', 'Occasionally', 'Regularly']]  # Define in increasing order

# Initialize and fit OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=categories)

# Replace the existing 'Alcohol Consumption' column with its encoded values
data['Alcohol Consumption'] = ordinal_encoder.fit_transform(data[['Alcohol Consumption']])

# Display first few rows
print(data.head())




In [ ]:
categoriess = [['Low', 'Medium', 'High']]

ordinal_encoder = OrdinalEncoder(categories=categoriess)

# Replace the existing 'Alcohol Consumption' column with its encoded values

data['Stress Levels'] = ordinal_encoder.fit_transform(data[['Stress Levels']])

# Display first few rows
print(data.head())





**MODEL TRAINING**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import numpy as np

# Example data (replace with your actual data)
# data = ...

x = data.iloc[:, [1,3,4]]  # Select the 2nd and 4th columns for features
y = data.iloc[:, [-1]]    # Select the last column for the target

# Create classifiers
cf1 = LogisticRegression()
cf2 = KNeighborsClassifier()
cf3 = RandomForestClassifier()

# List of estimators
esti = [('lr', cf1), ('knn', cf2), ('rfc', cf3)]

# Cross-validation and printing results
for est in esti:
    cv = cross_val_score(est[1], x, y.to_numpy().ravel(), cv=10, scoring='accuracy')
    print(est[0], np.round(np.mean(cv), 2))


from sklearn.ensemble import VotingClassifier
vc=  VotingClassifier(estimators = esti, voting = 'hard')
nv = cross_val_score(vc, x,y.to_numpy().ravel(),cv=10, scoring= 'accuracy')

print(nv)
    
    


In [ ]:
nv.mean()

In [ ]:
data

In [ ]:
data_new = data.iloc[:, [1,3,-6,-1]];
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# Initialize LabelEncoder

le = LabelEncoder();
data_new['Genetic Risk Factor (APOE-ε4 allele)'] = le.fit_transform(data_new['Genetic Risk Factor (APOE-ε4 allele)'])

In [ ]:
data_new

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# Assuming you have a dataframe `data` and the target column is 'Alzheimer’s Diagnosis'
# For example, let's use a dataset like 'data' as a placeholder.
# Replace 'data' with your actual dataset.

# Example data loading (you can replace this with your actual data)
# data = pd.read_csv('your_data.csv')  # Replace with your dataset loading method

# Corrected sampling and column drop
sampled_data = data_new.sample(400, replace=True)
X = sampled_data.drop(columns=['Alzheimer’s Diagnosis'])
y = sampled_data['Alzheimer’s Diagnosis']

# Create a Decision Tree classifier
dt = DecisionTreeClassifier()

# Perform cross-validation (10-fold) and calculate accuracy
cv_scores = cross_val_score(dt, X, y, cv=10, scoring='accuracy')

# Print the mean accuracy
print(np.mean(cv_scores))


In [ ]:
from sklearn.ensemble import BaggingClassifier
bg = BaggingClassifier(
     estimator=DecisionTreeClassifier(),
    n_estimators=200,
    max_samples=0.5,
    bootstrap=True,
    random_state=42
);

cvv= cross_val_score(bg, X, y, cv=10, scoring='accuracy')

print(np.mean(cvv))

In [ ]:
Applied algorithm on two different variants of the data but the accuracy score turned out to be similar